이 자료는 Hugging Face Pipeline API를 이용한 챗봇 튜토리얼로 작성되었습니다.  
2025년 9월 14일에 정상 동작을 확인하였습니다.  

1. Hugging Face 회원가입(Sign Up)

    - [Hugging Face 바로가기: https://huggingface.co/](https://huggingface.co/)  
    - Sign Up 따라하기  

2. Hugging Face 소개
   - Hugging Face(https://huggingface.co/): 누구나 쉽게 공유활용 가능한 AI 모델 허브 & 오픈소스 생태계임  
   - Hugging Face Hub 구조  
       - Model : NLP, CV, Multimodal 등 수십만 개의 사전학습 모델 저장소  
       - Datasets : 전처리된 대규모 공개 데이터셋  
       - Spaces : Gradio 또는 Streamlit 기반의 AI 데모 앱 호스팅 플랫폼
    - Hugging Face Pipeline API  
        - 자주 쓰이는 AI 태스크를 몇 줄로 실행 가능하게 만든 고수준 API  
        - 태스크 이름만 지정하면 토크나이저/모델 로딩+전처리+후처리 실행  
        - 초보자도 쉽게 사용 가능하며 프로토타입 데모 제작에 최적화    

3. Hugging Face의 Access Token 생성  

    - 우측상단 (三) 메뉴 -> Settings -> Tokens -> +Create new token
    - 읽기 전용(Read) / 쓰기(Write) / 관리자(Admin) 중 목적에 맞게 발급  
    - 발급된 코드는 다시 볼 수 없기 때문에 별도로 보관  

In [3]:
from huggingface_hub import login
# login(token="hf_xxx_your_token")

4. Transformer 설치  

In [4]:
!pip install --upgrade transformers

5. Colab에서 Hugging Face의 Access Token을 이용하여 로그인(LogIn)  

In [5]:
!hf auth login --token token --add-to-git-credential

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py", line 1782, in whoami
    hf_raise_for_status(r)
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 482, in hf_raise_for_status
    raise _format(HfHubHTTPError, str(e), response) from e
huggingface_hub.errors.HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2 (Request ID: Root=1-68c6c369-6f80ec3d4001

6. Git credential 설정 하기  

In [6]:
!git config --global credential.helper store

7. Hugging Face 로그인 상태 확인  

In [7]:
!hf auth whoami

gislee


8. 한국어 문장 요약 - Hugging Face pipeline API 이용 통합 로드

In [12]:
from transformers import pipeline

# pipeline을 사용하여 KoT5 요약 모델 로드
# 모델 이름과 리비전을 명시하는 것이 좋습니다.
summarizer_pipeline = pipeline(
    "summarization",
    model="psyche/KoT5-summarization"
    # KoT5 모델의 특정 리비전을 사용하고 싶다면 revision="..." 추가
)
input_text_for_pipeline = "오늘은 허깅페이스를 배우는 날입니다. 파이프라인을 활용하면 요약, 번역, 질의응답 같은 다양한 작업을 쉽게 수행할 수 있습니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다."
print("\n입력문장: \n"+input_text_for_pipeline)

# pipeline을 사용하여 요약 생성
# max_length 대신 max_new_tokens 사용 가능 (모델에 따라 다름)
# KoT5는 max_length가 더 적합할 수 있습니다.
summary_result_pipeline = summarizer_pipeline(
    input_text_for_pipeline,
    max_length=100, # 생성될 요약문의 최대 길이
    min_length=10,  # 생성될 요약문의 최소 길이
    do_sample=False # 샘플링 사용 여부
)

# 결과 출력
print("\n요약문장:")
print(summary_result_pipeline[0]['summary_text'])

Device set to use cuda:0
Your max_length is set to 100, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



입력문장: 
오늘은 허깅페이스를 배우는 날입니다. 파이프라인을 활용하면 요약, 번역, 질의응답 같은 다양한 작업을 쉽게 수행할 수 있습니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다.

요약문장:
파이프라인을 활용하면 요약, 번역, 질의응답 같은 작업을 쉽게 수행할 수 있습니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다.
